<a href="https://colab.research.google.com/github/LordHandLee/HonCSC499_detect_falls_soccer/blob/main/ReconstructedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import glob

import matplotlib.pyplot as plt
import cv2
from skimage import data,io
import matplotlib.image as mpimg

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

In [ ]:
base_model = keras.applications.efficientnet.EfficientNetB1(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3),
)


In [ ]:
base_model.trainable = True

In [ ]:
inputs = keras.Input(shape=(150,150,3))
#x = base_model(inputs, training=False)

x = tf.cast(inputs, tf.int32)

x = keras.layers.Normalization()(x)

x = keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2)(x)

x = keras.layers.RandomFlip()(x)

x = keras.layers.Resizing(height=224, width=224)(x)

x = base_model(x, training=True)

x = keras.layers.GlobalAveragePooling2D()(x)

x = keras.layers.Dense(1)(x)

#dropout?

outputs = keras.layers.Activation('sigmoid')(x)

model = keras.Model(inputs, outputs)




#  random_flip (RandomFlip)    (None, 150, 150, 3)       0

#  resizing (Resizing)         (None, 224, 224, 3)       0

#  efficientnetb1 (Functional  (None, None, None, 1280   6575239
#  )                           )

#  global_average_pooling2d (  (None, 1280)              0
#  GlobalAveragePooling2D)

#  dense (Dense)               (None, 1)                 1281

#  classification_head_1 (Ac



In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 tf.cast_3 (TFOpLambda)      (None, 150, 150, 3)       0         
                                                                 
 normalization_6 (Normaliza  (None, 150, 150, 3)       7         
 tion)                                                           
                                                                 
 random_translation_3 (Rand  (None, 150, 150, 3)       0         
 omTranslation)                                                  
                                                                 
 random_flip_3 (RandomFlip)  (None, 150, 150, 3)       0         
                                                                 
 resizing_3 (Resizing)       (None, 224, 224, 3)       0   

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss=keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=30)

In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
data_dir = "/content/drive/MyDrive/honor_thesis/image_depo/"

In [ ]:
datagen = ImageDataGenerator(rescale=1/255.0, validation_split=0.3)
width = 150
height = 150
trainDatagen = datagen.flow_from_directory(directory=data_dir, target_size=(width, height),
                                           class_mode='binary',
                                           batch_size=16,
                                           subset='training')
valDatagen = datagen.flow_from_directory(directory=data_dir, target_size=(width, height),
                                           class_mode='binary',
                                           batch_size=16,
                                           subset='validation')

Found 611 images belonging to 2 classes.
Found 260 images belonging to 2 classes.


In [ ]:
ls "/content/drive/MyDrive/honor_thesis/image_depo/"

fallen_images/  model_autokeras.h5  model_autokeras.keras  standing_images/


In [ ]:
history = model.fit(trainDatagen, steps_per_epoch = len(trainDatagen),
                    epochs=500, validation_data = valDatagen,
                    validation_steps = len(valDatagen),
                    callbacks=[early_stop])

Epoch 1/500


ValueError: ignored

In [ ]:
history = model.fit(trainDatagen, steps_per_epoch = len(trainDatagen),
                    epochs=500, validation_data = valDatagen,
                    validation_steps = len(valDatagen),
                    callbacks=[early_stop])

Epoch 1/500
39/39 [==============================] - 63s 152ms/step - loss: 0.9531 - binary_accuracy: 0.5123 - val_loss: 0.7326 - val_binary_accuracy: 0.5000
Epoch 2/500
39/39 [==============================] - 4s 89ms/step - loss: 0.7088 - binary_accuracy: 0.5548 - val_loss: 0.7354 - val_binary_accuracy: 0.4962
Epoch 3/500
39/39 [==============================] - 4s 90ms/step - loss: 0.7076 - binary_accuracy: 0.5417 - val_loss: 0.7012 - val_binary_accuracy: 0.5038
Epoch 4/500
39/39 [==============================] - 3s 87ms/step - loss: 0.7132 - binary_accuracy: 0.5106 - val_loss: 0.6888 - val_binary_accuracy: 0.5462
Epoch 5/500
39/39 [==============================] - 3s 88ms/step - loss: 0.6692 - binary_accuracy: 0.5679 - val_loss: 0.7334 - val_binary_accuracy: 0.5538
Epoch 6/500
39/39 [==============================] - 3s 89ms/step - loss: 0.6771 - binary_accuracy: 0.5646 - val_loss: 0.6916 - val_binary_accuracy: 0.5538
Epoch 7/500
39/39 [==============================] - 4s 89ms/s